# Step 1

In [34]:
import os
import shutil
import csv

# Mappatura delle estensioni dei file -- ho cercato su internet le estensioni per ciascuna tipologia di file e creato un dizionario
EXTENSIONS = {
    'audio': [".aac", ".aif", ".aiff", ".alac", ".amr", ".ape", ".flac", ".m4a", ".mp3", ".mpc", ".ogg", ".opus", ".ra", ".ram", ".wav", ".wma", ".3gp", ".audiobook", ".mid", ".midi", ".dsd", ".cda"],
    'doc': [".txt", ".doc", ".docx", ".odt", ".rtf", ".pdf", ".md", ".tex", ".log", ".csv", ".tsv", ".xml", ".json", ".yaml", ".html", ".css", ".ini", ".cfg"],
    'image': [".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp", ".svg", ".heif", ".heic", ".ico", ".raw", ".nef", ".cr2", ".dng", ".eps", ".ai", ".psd"]
}

# Funzione per determinare il tipo di file
def get_file_info(file_path):
    file_name, ext = os.path.splitext(os.path.basename(file_path))
    ext = ext.lower()

    for file_type, extensions in EXTENSIONS.items():
        if ext in extensions:
            return [file_name, file_type, os.path.getsize(file_path)]
    return [file_name, 'other', os.path.getsize(file_path)]

# Funzione per aggiungere le informazioni al file recap.csv
def update_recap(file_info):
    recap_path = './files/recap.csv'
    
    # Scrivere l'intestazione solo se il file è nuovo
    if not os.path.exists(recap_path):
        with open(recap_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['File Name', 'Type', 'SizeB'])
    
    # Aggiungere i dati al file riga a riga 
    with open(recap_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(file_info)

# Funzione per spostare il file considerando la casistica che un file con lo stesso nome potrebbe già esistere nella cartella
def move_file_with_rename(src_path, dest_dir):
    # Estrarre il nome e l'estensione del file
    base_name = os.path.basename(src_path)
    name, ext = os.path.splitext(base_name)
    
    # Costruirei il percorso completo di destinazione
    dest_path = os.path.join(dest_dir, base_name)
    
    # Controllare se il file esiste già nella destinazione
    counter = 1
    new_base_name = name
    while os.path.exists(dest_path):
        # Crea un nuovo nome aggiungendo un suffisso numerico tra parentesi
        new_name = f"{name}({counter}){ext}"
        new_base_name = f"{name}({counter})"
        dest_path = os.path.join(dest_dir, new_name)
        counter += 1
    
    # Spostare e rinominare il file
    shutil.move(src_path, dest_path)
    return new_base_name


# Creare le directory se non esistono
for directory in ["audio", "docs", "images"]:
    os.makedirs(os.path.join('files', directory), exist_ok=True)

# Ottenere l'elenco dei file nella cartella 'files' ordinati alfabeticamente e non considerando il file di recap che è un file di testo
files_directory = './files'
files = [item for item in os.listdir(files_directory) if os.path.isfile(os.path.join(files_directory, item))]
files.sort()
if 'recap.csv' in files:
    files.remove('recap.csv')

# Ciclo for per spostare i file
for file in files:
    file_path = os.path.join(files_directory, file)
    file_info = get_file_info(file_path)

    if file_info[1] != 'other':  # Se il file è audio, doc o immagine
        # Mappatura dei tipi di file per usare le cartelle al plurale (da esercizio la tipologia dei file e il nome delle cartelle non corrispondono)
        folder_mapping = {
            'audio': 'audio',    # "audio" -> "audio" 
            'doc': 'docs',       # "doc" -> "docs"
            'image': 'images'    # "image" -> "images"
        }

        # Spostamento del file nella cartella corrispondente
        target_dir = folder_mapping[file_info[1]] 
        target_path = os.path.join(files_directory, target_dir)
        file_info[0] = move_file_with_rename(file_path, target_path)

        # Stampare le info dei file che vengono spostai
        print(f"{file_info[0]} type:{file_info[1]} size:{file_info[2]}B")
        # Aggiornare il recap
        update_recap(file_info)

paperino(2) type:doc size:8299


# Step 3

In [54]:
from tabulate import tabulate
from PIL import Image
import numpy as np
import os # Lo riimporto immaginando che le due parti non lavorino assieme

# Funzione per formattare i numeri con due decimali
def format_number(num):
    num = float(num)
    return f"{num:.2f}"

# Elenco dei file nella cartella 'images' ordinati alfabeticamente
files_directory = './files/images'  
files = [item for item in os.listdir(files_directory) if os.path.isfile(os.path.join(files_directory, item))]
files.sort()

# Lista che raccoglierà tutte le righe della tabella
table_rows = []

for file_name in files:
    # Costruire il percorso completo dell'immagine
    image_path = os.path.join(files_directory, file_name)

    # Rimuovere l'estensione
    name_without_extension, _ = os.path.splitext(file_name) # '_' è una variabile di scarto
    
    # Aprire l'immagine e convertila in array con NumPy
    with Image.open(image_path) as im:
        im_arr = np.array(im)
        
    # Controllare il numero di canali (1, 3 o 4): 1 = scala di grigi, 3 = RGB, 4 = RGBA
    if len(im_arr.shape) == 2:  # Scala di grigio
        grayscale_mean = np.mean(im_arr)
        row = [name_without_extension, im_arr.shape[0], im_arr.shape[1],  format_number(grayscale_mean), format_number(0.00), format_number(0.00), format_number(0.00), format_number(0.00)]
    elif im_arr.shape[2] == 3:  # RGB
        r_mean = np.mean(im_arr[:, :, 0])
        g_mean = np.mean(im_arr[:, :, 1])
        b_mean = np.mean(im_arr[:, :, 2])
        row = [name_without_extension, im_arr.shape[0], im_arr.shape[1], format_number(0.00), format_number(r_mean), format_number(g_mean), format_number(b_mean), format_number(0.00)]
    else:  # RGBA
        r_mean = np.mean(im_arr[:, :, 0])
        g_mean = np.mean(im_arr[:, :, 1])
        b_mean = np.mean(im_arr[:, :, 2])
        alpha_mean = np.mean(im_arr[:, :, 3])
        row = [name_without_extension, im_arr.shape[0], im_arr.shape[1], format_number(0.00), format_number(r_mean), format_number(g_mean), format_number(b_mean), format_number(alpha_mean)]
    
    # Aggiungere la riga alla tabella
    table_rows.append(row)

# Stampare la tabella
headers = ["name", "height", "width", "grayscale", "R", "G", "B", "ALPHA"]
print(tabulate(table_rows, headers=headers, tablefmt="grid"))


+----------+----------+---------+-------------+--------+--------+-------+---------+
| name     |   height |   width |   grayscale |      R |      G |     B |   ALPHA |
+==========+==========+=========+=============+========+========+=======+=========+
| bw       |      512 |     512 |       21.48 |   0    |   0    |  0    |    0    |
+----------+----------+---------+-------------+--------+--------+-------+---------+
| daffodil |      500 |     335 |        0    | 109.25 |  85.56 |  4.97 |    0    |
+----------+----------+---------+-------------+--------+--------+-------+---------+
| eclipse  |      256 |     256 |        0    | 109.05 | 109.52 | 39.85 |  133.59 |
+----------+----------+---------+-------------+--------+--------+-------+---------+
| trump    |      183 |     275 |        0    |  97.01 |  98.99 | 90.92 |    0    |
+----------+----------+---------+-------------+--------+--------+-------+---------+
